In [ ]:
import numpy as np
import scipy.io as sio
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from eeg_emg_load import load_emg_eeg_data, butter_bandpass_filter, epoch, plot_epochs
from model_utils import train_LDA, train_RFs, train_SVM, train_HTNet, fold_split_data, balance_data

In [ ]:
if os.environ.get("CUDA_VISIBLE_DEVICES") is None:
    #Choose GPU 0 as a default if not specified (can set this in Python script that calls this)
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load in the EEG and EMG data

In [ ]:
path = '/data1/users/zsteineh/EE560';

eeg_folder = 'EEG'
eeg_file = 'EEG_session1_sub1_multigrasp_realMove.mat'

emg_folder = 'EMG'
emg_file = 'EMG_session1_sub1_multigrasp_realMove.mat'

# Load data in
emg_data, emg_fs, emg_event_times, emg_grasp_labels, emg_grasp_names = load_emg_eeg_data(os.path.join(path,emg_folder, emg_file));
eeg_data, eeg_fs, eeg_event_times, eeg_grasp_labels, eeg_grasp_names = load_emg_eeg_data(os.path.join(path,eeg_folder, eeg_file));

In [ ]:
emg_data = butter_bandpass_filter(emg_data, 8, 30, emg_fs, order=5)
eeg_data = butter_bandpass_filter(eeg_data, 8, 30, emg_fs, order=5)

In [ ]:
emg_epochs = epoch(emg_data,emg_event_times, emg_fs, 1);
eeg_epochs = epoch(eeg_data,eeg_event_times, eeg_fs, 1);

In [ ]:
plot_epochs(emg_epochs, emg_fs, 1, 10) # First number is number of channels, Second number is number of epochs
plot_epochs(eeg_epochs, eeg_fs, 1, 10) # First number is number of channels, Second number is number of epochs

In [ ]:
" Now let's see if we can plot the subset for a specified label"
plot_epochs(emg_epochs[:,:,emg_grasp_labels == 3], emg_fs, 1, 10)

In [ ]:
# samples, channels, num events
emg_epochs.shape

In [ ]:
eeg_grasp_names = [i[0] for i in eeg_grasp_names]
eeg_grasp_names

In [ ]:
np.all(emg_grasp_labels == eeg_grasp_labels)

In [ ]:
trimmed_grasp_labels, trimmed_emg_epochs, trimmed_eeg_epochs = balance_data(emg_grasp_labels, emg_epochs, eeg_epochs)

Still need code for pulliing out the CSP features

# Classify with just EEG data
... for now

In [ ]:
folds = 10
cur_channel = 2
fold_accs = train_LDA(folds, cur_channel, eeg_epochs, eeg_grasp_labels, eeg_grasp_names)

note to self: do I need to make sure that classes are balanced in both datasets?

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with LDA on one EEG channel')
plt.show()

# Classify EMG with LDA

In [ ]:
folds = 10
cur_channel = 2

fold_accs = train_LDA(folds, cur_channel, emg_epochs, emg_grasp_labels, eeg_grasp_names)

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with LDA on one EMG channel')
plt.show()

# Next will try with both EMG and EEG together

In [ ]:
folds = 10
cur_channel = 0

emg_eeg_epochs = np.concatenate( (emg_epochs[:,cur_channel,:], eeg_epochs[:,cur_channel,:]), axis=0 )
emg_eeg_epochs = np.expand_dims(emg_eeg_epochs, axis = 1)

fold_accs = train_LDA(folds, cur_channel, emg_eeg_epochs, emg_grasp_labels, eeg_grasp_names)

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with LDA on one EMG and EEG channel')
plt.show()

# Now trying Classification with RFs

In [ ]:
folds = 10
cur_channel = 2

fold_accs = train_RFs(folds, cur_channel, emg_epochs, emg_grasp_labels, eeg_grasp_names)

Unbalanced with Rest data really messing with the RF model

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with RFs on one EMG channel')
plt.show()

In [ ]:
np.unique(emg_grasp_labels, return_counts = True)

In [ ]:
folds = 10
cur_channel = 6

fold_accs = train_RFs(folds, cur_channel, trimmed_emg_epochs, trimmed_grasp_labels, eeg_grasp_names)

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with RFs on one EMG channel')
plt.show()

Not doing so well, but I'm guessing this could be because of the reading from one channel

In [ ]:
folds = 10
cur_channel = 0

fold_accs = train_RFs(folds, cur_channel, trimmed_eeg_epochs, trimmed_grasp_labels, eeg_grasp_names)

In [ ]:
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with RFs on one EEG channel')
plt.show()

# Trying with SVM

In [ ]:
folds = 10
cur_channel = 0

fold_accs = train_SVM(folds, cur_channel, trimmed_eeg_epochs, trimmed_grasp_labels, eeg_grasp_names)
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with SVM on one EEG channel')
plt.show()

# Train with HTNet

In [ ]:
folds = 10
cur_channel = 0

fold_accs = train_HTNet(folds, trimmed_eeg_epochs, trimmed_grasp_labels, eeg_grasp_names, eeg_fs)
plt.boxplot(np.array(fold_accs), labels=['Train', 'Test'])
plt.ylabel('Classification Accuracy')
plt.ylim((0,1))
plt.axhline((100/len(eeg_grasp_names))/100,c='k',linestyle='--')
plt.title('Classification Accuracy with HTNet on EEG data')
plt.show()

Getting something odd with the loss after training... This may have something to do with it: https://github.com/keras-team/keras/issues/6977